In [5]:
import evaluator_prob as evaluator
from qiskit.quantum_info.states.measures import state_fidelity
from qcg.generators import gen_supremacy, gen_hwea

circ = gen_supremacy(4,4,8,order='75601234')
sv1 = evaluator.simulate_circ(circ, 'statevector_simulator', noisy=False, provider_info=None, output_format='sv', num_shots=1024)
sv2 = evaluator.simulate_circ(circ, 'statevector_simulator', noisy=False, provider_info=None, output_format='sv', num_shots=1024)
state_fidelity(sv1,sv2)

0.999999999999994